In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from qiskit import QuantumCircuit, transpile, assemble, Aer
from qiskit.visualization import plot_histogram
from IPython.display import display

# Set a random seed for reproducibility
np.random.seed(42)

# Load the MNIST train and test datasets using pandas
mnist_train_df = pd.read_csv('mnist_train.csv')
mnist_test_df = pd.read_csv('mnist_test.csv')

# Extract features and target from the train dataset
X_train = mnist_train_df.drop('label', axis=1).values
y_train = mnist_train_df['label'].values

# Extract features and target from the test dataset
X_test = mnist_test_df.drop('label', axis=1).values
y_test = mnist_test_df['label'].values

# Define the feature_extraction_circuit function
def feature_extraction_circuit(x):
    circuit = QuantumCircuit(100, 1)  # Reduced number of qubits to 100
    for i in range(100):
        circuit.ry(x[i], i)
    circuit.measure(99, 0)
    return circuit

# Define the quantum feature extraction function
def quantum_feature_extraction(x):
    circuit = feature_extraction_circuit(x)
    return circuit

# Perform feature extraction on the training data
backend = Aer.get_backend('qasm_simulator')
batch_size = 100  # Adjust the batch size based on your system resources
X_train_quantum = []
for i in range(0, len(X_train), batch_size):
    batch = X_train[i : i + batch_size]
    qc_list = [quantum_feature_extraction(x) for x in batch]
    transpiled_circuits = transpile(qc_list, backend)
    qobj = assemble(transpiled_circuits, shots=1)
    results = backend.run(qobj).result().get_counts()
    features = [result.get('0', 0) / 1 for result in results.values()]
    X_train_quantum.extend(features)

# Perform feature extraction on the testing data
X_test_quantum = []
for i in range(0, len(X_test), batch_size):
    batch = X_test[i : i + batch_size]
    qc_list = [quantum_feature_extraction(x) for x in batch]
    transpiled_circuits = transpile(qc_list, backend)
    qobj = assemble(transpiled_circuits, shots=1)
    results = backend.run(qobj).result().get_counts()
    features = [result.get('0', 0) / 1 for result in results.values()]
    X_test_quantum.extend(features)

# Convert the feature vectors to NumPy arrays and reshape them
X_train_quantum = np.array(X_train_quantum).reshape(-1, 1)
X_test_quantum = np.array(X_test_quantum).reshape(-1, 1)

# Train the random forest classifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_quantum, y_train)

# Predict using the trained classifier
y_pred = rf.predict(X_test_quantum)

# Compute the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
